Segmenting and Clustering Neighborhoods in Toronto - Homework


This Notebook has been created to develop this homework.
First, we import and load the necessary libraries to carry out the entire process.

In [12]:
import requests
import lxml.html as lh
from itertools import cycle, islice
from matplotlib import colors
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import geocoder
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium 

In [13]:
#Download the data of the web
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [14]:
#Create page, to handle the contents of the website
page = requests.get(url)

In [15]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [16]:
#Now parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [17]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [18]:
tr_elements = doc.xpath('//tr')

In [19]:
#Create empty list
col=[]
i=0

In [20]:
#Store each first element (header) and an empty list, and print the headers
for t in tr_elements[0]:
    i+=1
    name=t.text_content()[:-1]
    print ("%d:%s"%(i,name))
    col.append((name,[]))

1:Postal Code
2:Borough
3:Neighbourhood


In [21]:
#The first row is the header, and data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()[:-1] 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [22]:
# Obtain the length of columns
[len(C) for (title,C) in col]

[181, 181, 181]

In [23]:
#Define the Data Frame
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [24]:
#Visualize the Data Frame
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


The data "Not assigned" in the column "Borough", and the last row, are dropped. Too, the index column is reseted.

In [25]:
df=df.drop(df[df["Borough"]=="Not assigned"].index)
df=df.drop(df[df["Borough"]=="Canadian postal codes"].index)
df.reset_index(drop=True,inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [26]:
df.shape

(103, 3)

In [27]:
#We save the Data Frame in a file .csv
df.to_csv('Toronto-Neighborhoods.csv')

In [43]:
#Reading a csv file with geospatial data of Toronto, and associate each postal code with your latitude and longitude.
TorontoGeo='http://cocl.us/Geospatial_data'
dfPoCoLaLo=pd.read_csv(TorontoGeo)
dfPoCoLaLo=dfPoCoLaLo.rename(columns={'Postal Code':'PostalCode'})
df=df.rename(columns={'Postal Code':'PostalCode'})
dfPoCoLaLo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
dfPoCoLaLo.shape
type(dfPoCoLaLo.iloc[0][0])

str

In [45]:
# We integrate the columns Latitude and Longitude with the data frame of neighborhoods of Toronto. 
df['Latitude']=None
df['Longitude']=None
df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",None,None
99,M4Y,Downtown Toronto,Church and Wellesley,None,None
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",None,None
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",None,None


In [46]:
# Now assigned the Latitude-Longitude information to each neighborhood according your postal code.
for i in df.index:
    for j in dfPoCoLaLo.index:
        if (df['PostalCode'][i]==dfPoCoLaLo['PostalCode'][j]):
            df['Latitude'][i]=dfPoCoLaLo['Latitude'][j]
            df['Longitude'][i]=dfPoCoLaLo['Longitude'][j]
df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6627,-79.3216
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985
